
---


## 🔍 ¿Qué problemas tiene **Simple Factory** que justifican migrar a **Factory Method**?

---

### 📦 1. **Acoplamiento a una fábrica concreta** DIP

En **Simple Factory**, el `PizzaStore` depende directamente de una instancia de `SimplePizzaFactory`. Eso significa que:

* Cada vez que querés cambiar cómo se crean las pizzas (por ejemplo, estilo Chicago), **tenés que cambiar o reemplazar la fábrica**.
* El store **no tiene control** sobre el proceso de creación, solo lo delega a una clase externa.

```python
store = PizzaStore(SimplePizzaFactory())
```

Esto **rompe el principio de inversión de dependencias (DIP)**: `PizzaStore` (nivel alto) depende de una clase concreta (`SimplePizzaFactory`) en lugar de una abstracción.

---

### 🧱 2. **No cumple Open-Closed Principle (OCP)**

Cada vez que querés agregar un nuevo tipo de pizza:

* Tenés que **modificar** la fábrica:

```python
def create_pizza(self, kind: str) -> Pizza:
    if kind == "cheese": return CheesePizza()
    elif kind == "pepperoni": return PepperoniPizza()
    # ⚠️ hay que tocar el código cada vez que se agrega una nueva pizza
```

Eso viola el **OCP**: el código debería estar abierto a extender (añadir nuevas pizzas), pero **cerrado a modificar**.

---

### 🌍 3. **No se adapta bien a contextos regionales/variantes**

Imaginá que tenés una `PizzaStore` en Nueva York y otra en Chicago. Cada una tiene su estilo:

* `NYStyleCheesePizza` vs `ChicagoStyleCheesePizza`
* Diferentes masas, salsas, formas de cortar la pizza...

Con Simple Factory:

* O tenés que **meter lógica condicional por región** dentro de la fábrica, lo cual escala mal.
* O creás **una fábrica distinta por región** (lo cual vuelve a generar acoplamiento y más código repetido).

---

### 🚫 4. **No se puede heredar el proceso de creación**

En Simple Factory:

* La lógica de creación está en una clase separada (`SimplePizzaFactory`).
* Si querés extender el proceso (por ejemplo, loguear o validar pedidos antes de crear), **no podés modificar `create_pizza` sin editar esa clase directamente**.

Con Factory Method:

* Esa lógica va en un método `create_pizza` que **cada subclase de `PizzaStore` puede sobrescribir**.

---

### 📌 En resumen:

| Problema                      | Simple Factory               | Por qué migrar a Factory Method                   |
| ----------------------------- | ---------------------------- | ------------------------------------------------- |
| Acoplamiento a clase concreta | Sí                           | Factory Method usa subclases → menos acoplamiento |
| Violación de OCP              | Sí, hay que modificar código | FM permite extender agregando subclases           |
| Mal manejo de variantes       | Sí, difícil de escalar       | FM se adapta bien a tiendas regionales            |
| Rígido en herencia/extensión  | Sí                           | FM permite heredar lógica de creación             |
| No cumple DIP                 | No                           | FM reduce dependencia directa a concretos         |

---

¿Querés que pasemos ahora a **refactorizar tu código actual para migrar a Factory Method**?
Puedo ayudarte a armar `PizzaStore` como clase abstracta y crear `NYPizzaStore` y `ChicagoPizzaStore` paso a paso.


Exacto ✅, lo entendiste bien. Te explico con un poquito más de detalle:

---

### 1️⃣ Cada **Store es un Factory**

* En Simple Factory, tenías algo como `SimplePizzaFactory.create_pizza(kind)` para **todas** las pizzas.
* En Factory Method, cada `PizzaStore` tiene **su propio método `create_pizza`**.

  * Ej: `NYPizzaStore.create_pizza("cheese")` → devuelve `NYStyleCheesePizza`
  * `ChicagoPizzaStore.create_pizza("cheese")` → devuelve `ChicagoStyleCheesePizza`

Esto hace que cada tienda sea responsable de **su propia familia de pizzas**. No necesitás una fábrica externa.

---

### 2️⃣ Preparación de pizzas

Cuando llamás:

```python
p = ny.order_pizza("cheese")
```

Pasa esto:

1. `order_pizza` llama a `create_pizza("cheese")`
2. `create_pizza` devuelve un objeto pizza específico (`NYStyleCheesePizza`)
3. Luego, `order_pizza` hace:

   ```python
   pizza.prepare()
   pizza.bake()
   pizza.cut()
   pizza.box()
   ```

   * Cada pizza puede tener su propio comportamiento si es necesario (por ejemplo, `ChicagoStyleCheesePizza.cut()` corta en cuadrados).

---

### 3️⃣ Resumen visual

```
Main
 └─ NYPizzaStore.order_pizza("cheese")
     └─ NYPizzaStore.create_pizza("cheese") → NYStyleCheesePizza
         └─ pizza.prepare()
         └─ pizza.bake()
         └─ pizza.cut()
         └─ pizza.box()
```

Y lo mismo para Chicago, pero con sus propias pizzas.

---

Si querés, puedo hacer un **diagrama rápido de cómo fluyen las llamadas en Factory Method** para que quede clarísimo la diferencia con Simple Factory. Eso hace que se vea “todo en acción”.

¿Querés que haga eso?


Perfecto, vamos a comparar **Simple Factory** vs **Factory Method** para ver pros y contras.

---

## **Beneficios de Factory Method respecto a Simple Factory**

1. **Desacoplamiento**

   * Cada `PizzaStore` sabe **cómo crear sus propias pizzas**, no depende de un único `SimplePizzaFactory`.
   * Esto hace que el código sea más **flexible** y **mantenible**.

2. **Extensible**

   * Querés agregar un nuevo estilo de pizza (por ejemplo, `CaliforniaPizzaStore`) → solo creás una nueva clase de tienda y definís su `create_pizza`.
   * No tenés que tocar una fábrica central y arriesgar romper otras pizzas.

3. **Especialización**

   * Cada pizza puede comportarse diferente según la tienda (ej: corte de pizza, toppings exclusivos) sin ensuciar la fábrica global.

4. **Cumple principios SOLID**

   * Principio de responsabilidad única: cada tienda es responsable de sus pizzas.
   * Principio abierto/cerrado: podés agregar nuevas tiendas sin modificar las existentes.

---

## **Cosas malas / desventajas**

1. **Más clases**

   * Necesitás una clase `PizzaStore` por cada “familia” de pizzas → más archivos, más boilerplate.

2. **Complejidad**

   * Para proyectos pequeños puede ser “demasiado” si tenés pocas pizzas.
   * Simple Factory era más directo y simple de entender.

3. **Difícil de centralizar**

   * Si querés ver **todas las pizzas disponibles** de todas las tiendas, no hay un solo lugar donde estén definidas, a diferencia del Simple Factory.

---

💡 **Regla práctica:**

* **Simple Factory:** bien para proyectos pequeños, pocas variaciones, rápido de implementar.
* **Factory Method:** necesario si tenés familias de productos diferentes, quieres extensibilidad y personalización por “tienda”.

---

Si querés, puedo hacerte un **cuadro comparativo simple** con “qué conviene cuándo” para que lo tengas visual. Eso ayuda mucho para la parte teórica de la materia.

¿Querés que haga eso?


¡Exacto! 💡 En la versión con **Abstract Factory**, se da un paso más allá: **no solo se abstrae *qué* pizza crear (Factory Method), sino también *cómo están hechas por dentro* (ingredientes)**.

---

## 🎯 ¿Qué resuelve la **Abstract Factory**?

Hasta Factory Method, vos decidís **qué tipo de pizza crear** (`CheesePizza`, `VeggiePizza`, etc.).

Pero… ¿qué pasa cuando las **familias de ingredientes cambian por región**?

* NY usa `ThinCrustDough`, `MarinaraSauce`, `ReggianoCheese`.
* Chicago usa `ThickCrustDough`, `PlumTomatoSauce`, `Mozzarella`.

➡️ Esos ingredientes son **clases concretas**. Si están embebidos directamente en las clases de pizza, terminás con esto:

```python
# NYCheesePizza
self.dough = ThinCrustDough()
self.sauce = MarinaraSauce()

# ChicagoCheesePizza
self.dough = ThickCrustDough()
self.sauce = PlumTomatoSauce()
```

⚠️ Problema: ¡Las clases de pizza **están acopladas a ingredientes concretos**! Esto viola DIP y OCP.

---

## ✅ Solución con Abstract Factory

### 🧩 Idea central:

* Crear una interfaz/fábrica abstracta llamada `PizzaIngredientFactory`, con métodos como:

  ```python
  def create_dough(self): ...
  def create_sauce(self): ...
  def create_cheese(self): ...
  ```

* Cada región implementa esa fábrica:

  * `NYPizzaIngredientFactory`
  * `ChicagoPizzaIngredientFactory`

* Las pizzas **ya no crean los ingredientes directamente**, sino que **se los piden a la fábrica** que reciben en su constructor.

---

### 💡 ¿Cómo se ve esto en la práctica?

#### 1. Interfaz de ingredientes (abstracta):

```python
class PizzaIngredientFactory(Protocol):
    def create_dough(self) -> Dough: ...
    def create_sauce(self) -> Sauce: ...
    def create_cheese(self) -> Cheese: ...
```

#### 2. Fábrica concreta (ej: NY):

```python
class NYPizzaIngredientFactory(PizzaIngredientFactory):
    def create_dough(self): return ThinCrustDough()
    def create_sauce(self): return MarinaraSauce()
    def create_cheese(self): return ReggianoCheese()
```

#### 3. Clase de pizza:

```python
class CheesePizza(Pizza):
    def __init__(self, ingredient_factory: PizzaIngredientFactory):
        self.ingredient_factory = ingredient_factory

    def prepare(self):
        print(f"Preparing {self.name}")
        self.dough = self.ingredient_factory.create_dough()
        self.sauce = self.ingredient_factory.create_sauce()
        self.cheese = self.ingredient_factory.create_cheese()
```

➡️ Notá que `CheesePizza` **no sabe nada** de `ThinCrustDough` ni `MarinaraSauce`.
Está **completamente desacoplada** de la región.

---

## 📦 ¿Qué gana el diseño?

| Ventaja                                | ¿Por qué importa?                                       |
| -------------------------------------- | ------------------------------------------------------- |
| Consistencia regional                  | Cada fábrica devuelve ingredientes que combinan bien.   |
| Menor acoplamiento                     | Las pizzas no conocen clases concretas de ingredientes. |
| Reutilización del flujo de preparación | `prepare()` siempre hace lo mismo, pide ingredientes.   |
| Cumple DIP y OCP                       | Cambiás fábricas, no modificás pizzas.                  |

---

## 🧠 En resumen

> **Sí**, en Abstract Factory, se le pasa a la pizza una **fábrica de ingredientes** (`PizzaIngredientFactory`), y esta es responsable de **crear los ingredientes específicos según la región**.
> Así, `Pizza.prepare()` ya no usa clases concretas, sino que delega esa responsabilidad.

---

¿Querés que armemos juntos un ejemplo mínimo de `PizzaIngredientFactory` y cómo se usa dentro de una `CheesePizza`?


Perfecto, gracias por compartir las salidas. Vamos a analizarlas parte por parte para que veas **cómo evolucionan los niveles de abstracción** desde Simple Factory hasta Abstract Factory, y cómo eso se refleja en el comportamiento del sistema.

---

## 🔍 Análisis de la salida

---

### ✅ 1. **Simple Factory**

```bash
Preparing Cheese Pizza
...
Ordered -> Cheese Pizza

Preparing Veggie Pizza
...
Ordered -> Veggie Pizza
```

🔎 **Observaciones:**

* Las pizzas son genéricas: `CheesePizza`, `VeggiePizza`, etc.
* No hay diferencia de región (NY o Chicago).
* Todos los ingredientes están **hardcodeados** en las clases de pizza (`Reggiano cheese`, `Mushroom`, etc.).
* La lógica de creación está encapsulada en una sola clase (`SimplePizzaFactory`), pero está **limitada a un solo estilo**.

🧠 **Conclusión:** Es un buen primer paso para desacoplar el `new`, pero no escala con variantes regionales ni cumple OCP/DIP.

---

### ✅ 2. **Factory Method**

```bash
******************** FACTORY_METHOD *************************
Preparing NY Style Sauce & Cheese
Adding toppings: Reggiano cheese
...

Preparing Chicago Style Deep Dish Cheese
Adding toppings: Shredded Mozzarella
Cutting the pizza into square slices
...
```

🔎 **Observaciones:**

* Las pizzas tienen nombre y preparación específicos por región:

  * NY usa **"Reggiano cheese"**
  * Chicago usa **"Shredded Mozzarella"**
* Cambia también la forma de cortar la pizza:

  * NY: diagonal
  * Chicago: **cuadrada**
* Las subclases `NYPizzaStore` y `ChicagoPizzaStore` implementan el método `create_pizza`, devolviendo clases concretas distintas como `NYStyleCheesePizza` o `ChicagoStyleCheesePizza`.

🧠 **Conclusión:**
Con Factory Method, **cada store crea sus propias variantes**, cumpliendo con el OCP. Pero todavía hay acoplamiento interno en las clases de pizza (ej. hardcodean `ReggianoCheese` o `Shredded Mozzarella` directamente).

---

### ✅ 3. **Abstract Factory**

```bash
************************* ABSTRACT *************************
Preparing NY Style Cheese Pizza
 -> Thin Crust Dough / Marinara Sauce / Reggiano Cheese
...

Preparing Chicago Style Clam Pizza
 -> Thick Crust Dough / Plum Tomato Sauce / Mozzarella Cheese / Frozen Clams
...
```

🔎 **Observaciones:**

* Se ven claramente los ingredientes específicos por región.
* Se usan **fábricas de ingredientes** (como `NYPizzaIngredientFactory`) para crear cada parte: masa, salsa, queso, almejas.
* La clase `CheesePizza` o `ClamPizza` **no conoce los ingredientes concretos**: se los pide a la fábrica.
* Esta es la implementación más **desacoplada y flexible**.

🧠 **Conclusión:**
Con Abstract Factory:

* Las pizzas ya no saben qué ingredientes usar: **solo saben que los obtienen de una fábrica**.
* Podés agregar nuevas regiones (ArgentinaPizzaFactory, TokyoPizzaFactory) sin tocar las clases de pizza.
* Cumple OCP y DIP perfectamente.

---

## 📌 Resumen comparativo

| Patrón           | Estilo de pizza | Diferencias regionales | Desacoplamiento ingredientes  | Escalabilidad |
| ---------------- | --------------- | ---------------------- | ----------------------------- | ------------- |
| Simple Factory   | Genérica        | ❌ No                   | ❌ No                          | Baja          |
| Factory Method   | Por región      | ✅ Sí (via subclases)   | ❌ No (ingredientes hardcoded) | Media         |
| Abstract Factory | Por región      | ✅ Sí                   | ✅ Sí (via fábrica)            | Alta          |

---

